# Notebook to analyse measured concentrations from MARGA
Read data from Excel files into Pandas dataframes

In [ ]:
# Import modules
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# Set some parameters
datadir = '/data/users/bdrummon/obs/nitrate_measurements_JanMay2020/'
auchencorth_filename = '2020filtered_Metoffice.xlsx'
chilbolton_filename1 = 'Chilbolton_JanMarch2020.csv'
chilbolton_filename2 = 'Prov_CHBO_MARGA_April-May_2020.xlsx'

### Function to load Auchencorth Moss data
A function to load data for the Auchencorth Moss Excel file, returns a dataframe

In [ ]:
def load_auchencorth_moss():
    
    mapper = {
        'Time_date' : 'time',
        'NH4_PM2_5' : 'PM2p5_NH4',
        'NO3_PM2_5' : 'PM2p5_NO3',
        'SO4_PM2_5' : 'PM2p5_SO4'
    }
    
    # Load file into dataframe
    df = pd.read_excel(datadir+auchencorth_filename, sheet_name='2020_filtered')
    
    # Get subset of columns
    df = df[['Time_date', 'NH4_PM2_5', 'NO3_PM2_5', 'SO4_PM2_5', 'NH3', 'HNO3', 'SO2']]
    
    # Rename columns
    df = df.rename(columns=mapper)
    
    # Drop the units row
    df = df.drop(0)
    
    # Convert string to datetime
    df['time'] = pd.to_datetime(df['time'])
    
    # Make datetime the index
    df = df.set_index('time')
    
    return df

### Functions to load Chilbolton data
- The Chilbolton data is split over two files - one downloaded from UK Air and one obtained directly from CEH (unratified data) - these are in different formats so we read them separately and then combine

In [ ]:
# Function to convert to datetime 
# We have to have a special way to handle midnight, which in the data is marked as 24:00:00
# Pandas can only handle hours 0 -> 23 so we have to convert 24:00:00 of day D to 00:00:00 of day D+1
def my_to_datetime(date_str):
    if date_str[10:12] != '24':
        return pd.to_datetime(date_str, format='%Y-%m-%d%H:%M:%S')

    date_str = date_str[0:10] + '00' + date_str[12:]
    return pd.to_datetime(date_str, format='%Y-%m-%d%H:%M:%S') + \
           dt.timedelta(days=1)

In [ ]:
def load_chilbolton_observatory1():
    
    mapper = {
        'ammonium in PM2.5' : 'PM2p5_NH4',
        'nitrate in PM2.5' : 'PM2p5_NO3',
        'sulphate in PM2.5' : 'PM2p5_SO4',
        'gaseous nitric acid' : 'HNO3',
        'gaseous ammonia' : 'NH3',
        'gaseous sulphur dioxide' : 'SO2'
    }
    
    # Load file into dataframe
    df = pd.read_csv(datadir+chilbolton_filename1, header=4, skipfooter=1)
    
    # Get subset of columns and rename
    df = df[['Date', 'Time']+list(mapper.keys())].rename(columns=mapper)
        
    # Combine date and time and convert to datetime
    df['time'] = df['Date'] + df['Time']
    df['time'] = df.time.apply(my_to_datetime)
    df = df.drop(columns=['Date', 'Time'])
    
    # Make time the index
    df = df.set_index('time')
    
    # Replace missing data with NaN
    df = df.replace('No data', np.nan)
    df = df.dropna()
    
    # Drop the last row (as this time is repeated in the April-May dataset)
    df = df[:-1]
    
    # Convert to float
    df = df.astype(float)
    
    return df

In [ ]:
def load_chilbolton_observatory2():
    
    mapper = {
        'm_NH4_2.5' : 'PM2p5_NH4',
        'm_NO3_2.5' : 'PM2p5_NO3',
        'm_SO4_2.5' : 'PM2p5_SO4',
        'm_HNO3_g' : 'HNO3',
        'm_NH3_g' : 'NH3',
        'm_SO2_g' : 'SO2'
    }
    
    # Load file into dataframe
    dfin = pd.read_excel(datadir+chilbolton_filename2, sheet_name='April_May_2020')
    
    # Create new dataframe
    df = pd.DataFrame()
    
    # Loop over requested species
    for species in mapper:
    
        # Get measurement times and measured concentrations for current species
        dfsp = dfin[dfin['parameter_id'] == species]
        dfsp = dfsp[['measurement start date', 'measurement start time', 'measurement']]
        
        # Combine the time and date columns and convert to datetime format
        dfsp['time'] = dfsp['measurement start date'].astype(str) + dfsp['measurement start time'].astype(str)
        dfsp['time'] = pd.to_datetime(dfsp['time'], format='%Y-%m-%d%H:%M:%S')
        
        # Drop redundant columns
        dfsp = dfsp.drop(columns=['measurement start date', 'measurement start time'])
        
        # Make time the index
        dfsp = dfsp.set_index('time')
        
        # Add new column to dataframe 
        df[mapper[species]] = dfsp['measurement']
        
        # Replace missing value flags (-999 in this dataset) with NaN
        mask = np.isclose(df[mapper[species]], -999.)
        df.loc[mask, mapper[species]] = np.nan
  
    return df

In [ ]:
def load_chilbolton_observatory():
    
    # Load January to March data
    df = load_chilbolton_observatory1()
    
    # Load April to May data
    df2 = load_chilbolton_observatory2()
    
    # Combine for a single timeseries
    df = df.append(df2)
    
    return df

# Load Chilbolton data and plot concentrations

In [ ]:
df = load_chilbolton_observatory()
df.plot()